# Assignmet 2
## Part 1 – Pandas DataFrames

1. Set your working directory and import the dataset Enaho01A-2023-300.csv using Pandas.

*Configuración del Directorio de Trabajo usando Rutas Relativas con ../

In [22]:
import os
import pandas as pd

# 1. Configurar el directorio de trabajo usando rutas relativas con ../
# Para subir 4 niveles desde tu ubicación actual y luego bajar a assingment_2/data
relative_path = "../../../../Documents/GitHub/assingment_2/data"

# 2. Convertir la ruta relativa a una ruta absoluta
absolute_path = os.path.abspath(relative_path)
print(f"Ruta absoluta calculada: {absolute_path}")

# 3. Verificar si la ruta existe
if not os.path.exists(absolute_path):
    print(f"Error: No se encuentra la ruta: {absolute_path}")
    print("Por favor, verifica la estructura de carpetas.")
else:
    # 4. Establecer el directorio de trabajo
    os.chdir(absolute_path)
    print(f"Directorio de trabajo establecido en: {os.getcwd()}")
    
    # 5. Importar el dataset Enaho01A-2023-300.csv
    try:
        # Intentar primero con ISO-8859-10 como se indica en la nota
        df = pd.read_csv("Enaho01A-2023-300.csv", encoding="ISO-8859-10", low_memory=False)
        print("Dataset importado con encoding ISO-8859-10")
    except UnicodeDecodeError:
        try:
            # Si ISO-8859-10 falla, intentar con UTF-8
            df = pd.read_csv("Enaho01A-2023-300.csv", encoding="UTF-8", low_memory=False)
            print("Dataset importado con encoding UTF-8")
        except Exception as e:
            print(f"Error: No se pudo importar el dataset: {e}")
            df = None

Ruta absoluta calculada: C:\Users\HOME\Documents\GitHub\assingment_2\data
Directorio de trabajo establecido en: C:\Users\HOME\Documents\GitHub\assingment_2\data
Dataset importado con encoding ISO-8859-10


*realizar las operaciones solicitadas

In [23]:
if df is not None:
    # a. Leer y mostrar las primeras 5 filas
    print("\nPrimeras 5 filas del dataset:")
    print(df.head())
    
    # b. Convertir los nombres de columna a una lista e imprimirla
    column_names = df.columns.tolist()
    print("\nNombres de columnas como lista:")
    print(column_names)
    
    # c. Verificar los tipos de datos del DataFrame
    print("\nTipos de datos del DataFrame:")
    print(df.dtypes)
    
    # d. Seleccionar un subconjunto con las variables requeridas
    variables_base = ['CONGLOME', 'VIVIENDA', 'HOGAR', 'CODPERSO']
    
    # Seleccionar 4 variables adicionales de interés educativo
    variables_adicionales = [
        'P300A',  # Idioma o lengua materna
        'P301A',  # Último año o grado de estudios y nivel que aprobó - Nivel
        'P302A',  # En los últimos 12 meses, ¿Recibió programa de alfabetización?
        'P304A'   # ¿Cuál es el grado o año de estudios al que asistió el año pasado? - Nivel
    ]
    
    # Verificar que las variables existan en el dataset
    variables_existentes = [col for col in variables_base + variables_adicionales if col in df.columns]
    
    # Crear el subconjunto
    subsample = df[variables_existentes]
    
    print(f"\nSubconjunto seleccionado con {len(variables_existentes)} variables:")
    print(subsample.head())
    
    # Mostrar información sobre el subconjunto
    print(f"\nDimensiones del subconjunto: {subsample.shape}")
    print("\nTipos de datos del subconjunto:")
    print(subsample.dtypes)
     # 7. Renombrar las variables adicionales para mayor claridad
    nuevos_nombres = {
        'P300A': 'lengua_materna',
        'P301A': 'nivel_educativo',
        'P302A': 'programa_alfabetizacion',
        'P304A': 'nivel_asistencia_actual'
    }
    
    subsample = subsample.rename(columns=nuevos_nombres)
    
    print("\nSubconjunto con nombres de variables renombrados:")
    print(subsample.head())
    



Primeras 5 filas del dataset:
    AŅO  MES  CONGLOME  VIVIENDA  HOGAR  CODPERSO  UBIGEO  DOMINIO  ESTRATO  \
0  2023    1      5030         2     11         1   10201        7        4   
1  2023    1      5030         2     11         2   10201        7        4   
2  2023    1      5030         2     11         3   10201        7        4   
3  2023    1      5030         2     11         4   10201        7        4   
4  2023    1      5030        11     11         1   10201        7        4   

   CODINFOR  ...  I311D$5  I311D$6  I311D$7  I3121C I3122C I315B    FACTOR07  \
0         1  ...                                                  118.374542   
1         2  ...                                                  118.374542   
2         2  ...                                                  118.374542   
3         2  ...                          8                       118.374542   
4         1  ...                                                  118.374542   

    FACTORA07

2. Data Manipulation (Data Cleaning):

In [38]:
def explore_and_clean_data(df):
    import pandas as pd
    import numpy as np
        # 1. Exploración inicial del DataFrame
    print("1. INFORMACIÓN GENERAL DEL DATAFRAME:")
    print(f"Dimensiones: {df.shape} (filas, columnas)")
    print(f"Número total de datos: {df.size}")
    
    # 2. Resumen estadístico
    print("\n2. RESUMEN ESTADÍSTICO DE VARIABLES NUMÉRICAS:")
    print(df.describe())
    
    # 3. Información sobre tipos de datos
    print("\n3. INFORMACIÓN SOBRE TIPOS DE DATOS:")
    print(df.info())
    
    # 4. Identificar valores missing
    print("\n4. IDENTIFICACIÓN DE VALORES MISSING:")
    missing_values = df.isnull().sum()
    missing_percentage = (df.isnull().sum() / len(df)) * 100
    
    missing_info = pd.DataFrame({
        'Valores_Missing': missing_values,
        'Porcentaje_Missing': missing_percentage
    })
    
    # Mostrar solo columnas con valores missing
    missing_info = missing_info[missing_info['Valores_Missing'] > 0]
    
    if len(missing_info) > 0:
        print("Columnas con valores missing:")
        print(missing_info)
    else:
        print("No se encontraron valores missing en el DataFrame.")
    
    # 5. Identificar valores codificados como missing (según diccionario ENAHO)
    print("\n5. IDENTIFICACIÓN DE VALORES CODIFICADOS COMO MISSING:")
    
    # Definir valores que representan missing según diccionario ENAHO
    missing_codes = {
        'P300A': [99],  # Lengua materna
        'P301A': [99],  # Nivel educativo
        'P301B': [99],  # Años de educación
        'P302A': [9],   # Programa de alfabetización
        'P304A': [9]    # Nivel de asistencia actual
    }
    